In [15]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
from utils import defaultCityDistricts, mkdir

import classroom
import teachers
import course
import aClass
import students
import meeting
import exams
import surveys
import attendance


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
#This cell generates data

df_classrooms = classroom.generate(357, 999)
df_teachers = teachers.generate(300)
df_courses = course.generate(300, df_teachers)
df_classes, df_courses = aClass.generate(10, df_teachers, df_courses)
df_students = students.generate(800, df_classes)
# if too many meetings have to be scheduled, may enter infinite loop; if it happens restart the kernel and run the code again
df_meetings = meeting.generate(0, df_courses, df_classrooms, df_classes, "2012-05-25", "2022-06-03")
df_exams = exams.generate(0, df_meetings, df_students)
df_surveys = surveys.generate(0, df_meetings, df_students, df_courses)
df_attendance = attendance.generate(0, df_meetings, df_students, df_teachers, df_courses)

Generating schedule for 8 days between 2022-05-25 and 2022-06-03
Starting at 2024-04-14 12:45:38.541453


c:\projects\schoolDataGeneration\meeting.py:189: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  dailySchedule = pd.concat([dailySchedule, classSchedule])


|2022-05-25|>>>|2022-05-25|>>>|2022-06-03|12.5%   ETC:0:00:11s
|2022-05-25|>>>|2022-05-26|>>>|2022-06-03|25.0%   ETC:0:00:09s
|2022-05-25|>>>|2022-05-27|>>>|2022-06-03|37.5%   ETC:0:00:08s
|2022-05-25|>>>|2022-05-30|>>>|2022-06-03|50.0%   ETC:0:00:06s
|2022-05-25|>>>|2022-05-31|>>>|2022-06-03|62.5%   ETC:0:00:05s
|2022-05-25|>>>|2022-06-01|>>>|2022-06-03|75.0%   ETC:0:00:03s
|2022-05-25|>>>|2022-06-02|>>>|2022-06-03|87.5%   ETC:0:00:02s
|2022-05-25|>>>|2022-06-03|>>>|2022-06-03|100.0%   ETC:0:00:00s


In [14]:
df_meetings[df_meetings["ClassName"] == "A22"]

,mID,Date,RoomNumber,cID,ClassName,StartHour,EndHour,Year,Month
0,0,2022-05-25 07:00:00,853,300,A22,7,8,2022,05
1,1,2022-05-25 08:00:00,383,238,A22,8,9,2022,05
2,2,2022-05-25 09:00:00,729,226,A22,9,10,2022,05
3,3,2022-05-25 10:00:00,921,149,A22,10,11,2022,05
4,4,2022-05-25 11:00:00,882,13,A22,11,12,2022,05
5,5,2022-05-25 12:00:00,272,294,A22,12,13,2022,05
6,6,2022-05-25 13:00:00,412,46,A22,13,14,2022,05
441,441,2022-05-26 07:00:00,359,300,A22,7,8,2022,05
442,442,2022-05-26 08:00:00,758,238,A22,8,9,2022,05
443,443,2022-05-26 09:00:00,297,226,A22,9,10,2022,05


In [55]:
df_meetings["ClassName"].unique()

array(['A22', 'A21', 'A20', 'A19', 'A18', 'A17', 'B22', 'B21', 'B20',
       'B19', 'B18', 'B17', 'C22', 'C21', 'C20', 'C19', 'C18', 'C17',
       'D22', 'D21', 'D20', 'D19', 'D18', 'D17', 'E22', 'E21', 'E20',
       'E19', 'E18', 'E17', 'F22', 'F21', 'F20', 'F19', 'F18', 'F17',
       'G22', 'G21', 'G20', 'G19', 'G18', 'G17', 'H22', 'H21', 'H20',
       'H19', 'H18', 'H17', 'I22', 'I21', 'I20', 'I19', 'I18', 'I17',
       'J22', 'J21', 'J20', 'J19', 'J18', 'J17'], dtype=object)

In [ ]:
#This cell loads data from csv files

path = "data"

df_classrooms = pd.read_csv(f"{path}/classrooms.csv")
df_teachers = pd.read_csv(f"{path}/teachers.csv")
df_courses = pd.read_csv(f"{path}/courses.csv")
df_classes = pd.read_csv(f"{path}/classes.csv")
df_students = pd.read_csv(f"{path}/students.csv")
df_meetings = pd.read_csv(f"{path}/meetings.csv")
df_exams = pd.read_csv(f"{path}/exams.csv")
df_surveys = pd.read_csv(f"{path}/surveys.csv")
df_attendance = pd.read_csv(f"{path}/attendance.csv")

In [4]:
#This cell purges unused rows
size =len(df_classrooms)
df_classrooms = df_classrooms[df_classrooms["RoomNumber"].isin(df_meetings["RoomNumber"])]
df_classrooms_removed = df_classrooms[~df_classrooms["RoomNumber"].isin(df_meetings["RoomNumber"])]
print(f"Removed {size - len(df_classrooms)} classrooms")

size =len(df_teachers)
df_teachers = df_teachers[df_teachers["tID"].isin(pd.concat([df_classes["tID"], df_courses["tID"]]).unique())]
df_teachers_removed = df_teachers[~df_teachers["tID"].isin(df_classes["tID"] + df_courses["tID"])]
print(f"Removed {size - len(df_teachers)} teachers")

size =len(df_courses)
df_courses = df_courses[df_courses["cID"].isin(df_classes["courses"].explode().unique())]
df_courses_removed = df_courses[~df_courses["cID"].isin(df_classes["courses"].explode().unique())]
print(f"Removed {size - len(df_courses)} courses")

size =len(df_classes)
df_classes = df_classes[df_classes["ClassName"].isin(df_students["ClassName"])]
df_courses_classes = df_classes[~df_classes["ClassName"].isin(df_students["ClassName"])]
print(f"Removed {size - len(df_classes)} classes")

Removed 47 classrooms
Removed 91 teachers
Removed 85 courses
Removed 0 classes


In [9]:
path = "data"
headers = True
indexing = False

mkdir(path)

df_students.drop(columns=["iq"]).to_csv(f"{path}/students.csv", index=indexing, header=headers)
df_classes.drop(columns=["Year", "courses"]).to_csv(f"{path}/classes.csv", index=indexing, header=headers)
df_teachers.to_csv(f"{path}/teachers.csv", index=indexing, header=headers)
df_courses.to_csv(f"{path}/courses.csv", index=indexing, header=headers)
df_classrooms.to_csv(f"{path}/classrooms.csv", index=indexing, header=headers)
df_meetings.drop(columns=["StartHour", "EndHour", "Year", "Month"]).to_csv(f"{path}/meetings.csv", index=indexing, header=headers)
df_surveys.to_csv(f"{path}/surveys.csv", index=indexing, header=headers)
df_exams.to_csv(f"{path}/exams.csv", index=indexing, header=headers)
df_attendance.to_csv(f"{path}/attendance.csv", index=indexing, header=headers)


Directory data already exists


In [ ]:
path = "data"
headers = True
indexing = False

mkdir(path)

df_students.to_csv(f"{path}/students.csv", index=indexing, header=headers)
df_classes.to_csv(f"{path}/classes.csv", index=indexing, header=headers)
df_teachers.to_csv(f"{path}/teachers.csv", index=indexing, header=headers)
df_courses.to_csv(f"{path}/courses.csv", index=indexing, header=headers)
df_classrooms.to_csv(f"{path}/classrooms.csv", index=indexing, header=headers)
df_meetings.to_csv(f"{path}/meetings.csv", index=indexing, header=headers)
df_surveys.to_csv(f"{path}/surveys.csv", index=indexing, header=headers)
df_exams.to_csv(f"{path}/exams.csv", index=indexing, header=headers)
df_attendance.to_csv(f"{path}/attendance.csv", index=indexing, header=headers)

In [12]:
path = "updates"

mkdir(path)

df = df_teachers.sample(n=15)
df["CityDistrict2"] = df["CityDistrict"].apply(lambda x: np.random.choice([y for y in defaultCityDistricts if y != x]))
df["update"] = df.apply(lambda x: f"Update Teachers SET CityDistrict = '{x['CityDistrict2']}' WHERE tID = {x['tID']};", axis=1)
df["update"].to_csv(f"{path}/updateTeachersCity.sql", index=False, header=False)

df = df_students.sample(n=15)
df["CityDistrict2"] = df["CityDistrict"].apply(lambda x: np.random.choice([y for y in defaultCityDistricts if y != x]))
df["update"] = df.apply(lambda x: f"Update Students SET CityDistrict = '{x['CityDistrict2']}' WHERE sID = {x['sID']};", axis=1)
df["update"].to_csv(f"{path}/updateStudentsCity.sql", index=False, header=False)

df = df_students.sample(n=15)
df["ClassName2"] = df["ClassName"].apply(lambda x: np.random.choice([y for y in df_classes["ClassName"].unique() if y != x]))
df["update"] = df.apply(lambda x: f"Update Students SET ClassName = '{x['ClassName']}' WHERE sID = {x['sID']};", axis=1)
df["update"].to_csv(f"{path}/updateStudentsClass.sql", index=False, header=False)

df = df_classes.sample(n=15)
df["tID2"] = df["tID"].apply(lambda x: np.random.choice([y for y in df_classes["ClassName"].unique() if y != x]))
df["update"] = df.apply(lambda x: f"Update Class SET tID = '{x['tID']}' WHERE ClassName = '{x['ClassName']}';", axis=1)
df["update"].to_csv(f"{path}/updateClassesTeacher.sql", index=False, header=False)

OSError: Cannot save file into a non-existent directory: 'updates'

In [41]:
schoolStart = 8
schoolEnd = 17
rooms = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

meeting.getAllRoomsCombinations(rooms, 3, 12)

{3: [6, 1, 4, 2, 8, 7, 9, 10, 5, 3],
 4: [2, 5, 10, 3, 9, 1, 7, 4, 8, 6],
 5: [8, 3, 5, 9, 1, 6, 10, 2, 4, 7],
 6: [2, 3, 6, 9, 5, 1, 7, 4, 10, 8],
 7: [4, 1, 10, 2, 3, 6, 5, 9, 7, 8],
 8: [4, 9, 5, 6, 2, 10, 3, 7, 8, 1],
 9: [7, 8, 3, 5, 4, 9, 1, 2, 10, 6],
 10: [3, 5, 10, 8, 7, 1, 2, 6, 9, 4],
 11: [9, 5, 8, 6, 4, 7, 10, 3, 2, 1],
 12: [7, 4, 10, 9, 1, 6, 8, 2, 3, 5]}

In [8]:
import random
import numpy as np

In [61]:
lista = [1,2,3,4,5]
zbior = set([3,4,5,6,7])
slownik = {1: "a", 2: "b", 3: "c", 4: "d", 5: "e"}
zbior.pop()

3

In [62]:
slownik.values()

dict_values(['a', 'b', 'c', 'd', 'e'])

In [63]:
import utils

In [64]:
utils.getKeyByValue(slownik, "c")

3